# RDF Conversions

This notebook demonstrates how to [convert](https://nexus-forge.readthedocs.io/en/latest/interaction.html#converting) data in any RDF format (n3, Turtle, ...) to Resources and vice-versa. 

In [ ]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [ ]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [ ]:
from kgforge.core import Resource

## Resource to rdflib.Graph

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.register(jane)

In [ ]:
graph = forge.as_graph(jane)

In [ ]:
len(graph) == 2

In [ ]:
for triple in graph:
    print(triple)

In [ ]:
graph_store_metadata = forge.as_graph(jane, store_metadata=True)
len(graph_store_metadata)

In [ ]:
for triple in graph_store_metadata:
    print(triple)

## rdflib.Graph to Resource

In [ ]:
#data in n3 format
data = '''
@prefix schema: <http://schema.org/> .
[] a schema:Person;
     schema:name "Jane Doe" ;
     schema:knows <http://example.org/JohnDoe>;
     schema:affiliation <https://www.grid.ac/institutes/grid.5333.6>.

<http://example.org/JohnDoe> a schema:Person;
     schema:name "John Doe" .
     
<https://www.grid.ac/institutes/grid.5333.6> a schema:Organization;
     schema:name "EPFL" .
'''

In [ ]:
import rdflib
graph = rdflib.Graph()
graph.parse(data=data, format="n3")
len(graph)

In [ ]:
resources = forge.from_graph(graph)

In [ ]:
len(resources) == 3

### Apply a JSON-LD Frame

In [ ]:
frame = {
        "@type": [ 'http://schema.org/Person'],
        "@embed": True
}  
resources = forge.from_graph(data=graph, frame= frame)

In [ ]:
len(resources) == 2

In [ ]:
forge.as_jsonld(resources)

### Select a Type

In [ ]:
_type= ["http://schema.org/Organization"]
resources = forge.from_graph(data=graph, type= _type)

In [ ]:
resources

In [ ]:
isinstance(resources, Resource)

In [ ]:
epfl_json = {'id': 'https://www.grid.ac/institutes/grid.5333.6',
  'type': 'schema:Organization',
  'schema:name': 'EPFL'}

In [ ]:
epfl_json == forge.as_json(resources)

### Use the forge Model context

In [ ]:
_type= ["http://schema.org/Organization"]
resources = forge.from_graph(data=graph, type= _type, use_model_context=True)

In [ ]:
isinstance(resources, Resource)